In [1]:
!which python

/Users/apabook/Desktop/gmv_rainbow/.venv/bin/python


In [8]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [9]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

In [10]:
def replace_host_docker_internal(url: str) -> str:
    return url.replace("host.docker.internal", "127.0.0.1")

### Provider and Consumer ids

In [11]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Consumer":
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
        "participant_slug": "Myself",
        "participant_type": "Provider",
        "base_url": "http://host.docker.internal:1200",
        "token": null,
        "saved_at": "2025-12-09T10:07:38.789516",
        "last_interaction": "2025-12-09T10:07:38.789516",
        "is_me": true
    }
]


In [12]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Provider":
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiJ4T0F3WlpkQ3AwQ0dCQV9tMVVHR1VXcEM5THJlcmJjbDMxblJoZThzNzd6R204aXA3YUgtUW5NNUNLbEo1YXlvUDhKTV9rLWhVQ0V2Q09qNWw5VmNhN0RfNTdRMjRJUEtHN0dQSGdnYzZ5bGxwZFlnSkVlY2xUcXJGcjdGUFBxcThDQzY1WVpfS1hGY3ByOHJGRVFEUGtkc3gzbU1qTkYzQUpNNzBpaEItdGtOOWxxcHZLb0h6MzJRaHRybVhFY2t1X1NtVFlFTU9GVU0xd0I1TE4xNmN4ZzIxZXYwMnZGdlVCYmdWZENTNHZocjNHNlNMM2dPZXVzd3BZaTZtTTZ6clloY3lkaDJaRjZCUEZVMS1NNk9tcENGRnd6SFVWNTdqMHJMLUR0OU1Ja1FUWmxXam5kcVVOZG1Ecnlyd0E1OU8zS1lWYWY5cmUwdW9QS3ZGUmRmY3cifQ",
        "participant_slug": "Myself",
        "participant_type": "Consumer",
        "base_url": "http://host.docker.internal:1100",
        "token": null,
        "saved_at": "2025-12-09T10:07:35.344175",
        "last_interaction": "2025-12-09T10:07:35.344182",
        "is_me": true
    },
    {
        "participant_id": "null",
        "participant_slug": "authority",
        "participant_type": "Authority",
        "base_url": "http://host.docker.

## Create Catalog instances and metadata

In [13]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:b3e01b82-0813-44de-80d5-dd5ba0455f8f'

In [14]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:44acff00-6386-4d88-99ee-ce63036c5056'

In [15]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:804e8297-1bce-4aaa-bb04-36da980bb888'

In [16]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

'urn:uuid:0e4c2e1f-eb06-4a8a-816b-27bc0037d6bd'

## Create policies

In [17]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:fa65db81-27d6-44eb-a4e7-669b09c62e74",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:44acff00-6386-4d88-99ee-ce63036c5056"
}


## Contract Negotiation

In [27]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
# cn_consumer_id = response_as_json["consumerPid"]
# cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "@context": [
    "https://w3id.org/dspace/2025/1/context.jsonld"
  ],
  "@type": "TransferError",
  "code": "TRANSFER_ERROR_CODE",
  "reason": [
    "Internal server error",
    "Not checked error.",
    "Error parsing mate: Missing Resource Error"
  ]
}


In [19]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "error": {
    "code": "400",
    "title": "BAD_REQUEST",
    "message": "JsonDataError: Failed to deserialize the JSON body into the target type: consumerPid: invalid urn scheme at line 1 column 47"
  }
}


In [20]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

KeyError: 'consumerPid'

In [21]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "error": {
    "code": "400",
    "title": "BAD_REQUEST",
    "message": "JsonDataError: Failed to deserialize the JSON body into the target type: consumerPid: invalid urn scheme at line 1 column 47"
  }
}


In [22]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "error": {
    "code": "400",
    "title": "BAD_REQUEST",
    "message": "JsonDataError: Failed to deserialize the JSON body into the target type: consumerPid: invalid urn scheme at line 1 column 47"
  }
}


In [23]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

KeyError: 'agreement'

In [24]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "error": {
    "code": "400",
    "title": "BAD_REQUEST",
    "message": "JsonDataError: Failed to deserialize the JSON body into the target type: consumerPid: invalid urn scheme at line 1 column 47"
  }
}


In [25]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "error": {
    "code": "400",
    "title": "BAD_REQUEST",
    "message": "JsonDataError: Failed to deserialize the JSON body into the target type: consumerPid: invalid urn scheme at line 1 column 47"
  }
}


## Transfer Negotiation


In [57]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
  "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
  "agreementId": "urn:uuid:5163f6e2-ec73-4542-b1eb-1fcc82967130",
  "callbackAddress": "http://host.docker.internal:1100/urn:uuid:45dfb269-a8f8-4c86-92dd-5d5e7ef22f72",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
    "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
    "state": "REQUESTED"
  }
}


In [58]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
  "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://host.docker.internal:1200/data/urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
    "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
    "state": "STARTED"
  }
}


In [59]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
consumer_data_address = replace_host_docker_internal(consumer_data_address)
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://host.docker.internal:1100/data/urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-10-23T13:25:28.834038",
  "updated_at": "2025-10-23T13:25:28.844682",
  "state": "STARTED"
}


In [60]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)


     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [62]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
  "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
    "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
    "state": "SUSPENDED"
  }
}


In [66]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [67]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "message": "Unexpected response from the Provider",
  "error_code": 2200,
  "details": null
}


In [68]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [69]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
  "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
    "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
    "state": "SUSPENDED"
  }
}


In [70]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
  "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:9b5da683-7210-497b-94bf-f8de247209e7",
    "consumerPid": "urn:uuid:39466a70-e81b-447d-8986-dcad56b1fbfd",
    "state": "COMPLETED"
  }
}
